# Model

Import necessary libraries.

In [8]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, Model

from sklearn.model_selection import train_test_split

Read the augmented data.

In [4]:
lm = pd.read_json('new_data/lm_300wLP_anno_tr.json')

## Building the Model

In [13]:
# Train-test split
X_train, X_test = train_test_split(lm['landmarks_2d'], test_size=0.2, random_state=42)
y_train, y_test = train_test_split(lm['landmarks'], test_size=0.2, random_state=42)

In [14]:
X_test

1113    [[[134.8625641, 254.3249359], [136.4567108, 27...
1309    [[[138.4798737, 267.6000977], [140.4720154, 29...
805     [[[137.0505981, 268.2783508], [140.3344421, 28...
1173    [[[136.6831818, 232.1711884], [139.3064575, 26...
2770    [[[140.2433472, 253.5360107], [144.2132263, 27...
                              ...                        
3798    [[[132.6191711, 247.0334015], [136.8818665, 27...
376     [[[143.4956818, 247.5625305], [137.6270905, 26...
1263    [[[154.2479553, 289.163147], [154.1784668, 307...
1075    [[[134.6793518, 230.1926422], [134.34729, 256....
2421    [[[137.6133881, 234.6465454], [138.1734314, 25...
Name: landmarks_2d, Length: 768, dtype: object